老师您好，我叫孟干，来自中央财经大学金融学院。
以下是我作业一的代码，请您查看。

In [ ]:
#1、调取相关数据分析库
import os
import pandas as pd
import numpy as np
import datetime
os.getcwd ()
import matplotlib.pyplot as plt

In [2]:
#2、读取数据库，为工作簿的第一个Sheet，并剔除其前三行，将第一列（日期列）设为索引列。
dataset=pd.read_excel("作业相关数据.xlsx","作业一",skiprows=[0,1,2],index_col=0,parse_dates=True)

In [3]:
#3、变更时间序列频率为月，并对空值进行填充，取空值前一天为该值。
dataset_monthly=dataset.asfreq("1M",method="ffill")

In [4]:
#4、读取中证800指数，存为变量equity_series；上证国债指数存为变量debt_series，设定无风险利率rf=1.5%。
equity_series=dataset_monthly.iloc[:,0]
debt_series=dataset_monthly.iloc[:,1]
rf=0.015

In [5]:
#5、构建波动率函数。
def volatility(series:pd.Series):
    return series.pct_change().std()

In [6]:
#6、构建累计收益率函数。
def cumulative_return(series:pd.Series):
    return series.dropna()[-1]/series.dropna()[0]-1

In [7]:
#7、构建计算投资年数函数。
def get_years(series:pd.Series):
    start_date=series.index[0]
    end_date=series.index[-1]
    return (end_date-start_date).days/365.25

In [8]:
#8、构建复合年化收益率函数。
def cagr(series: pd.Series):
    start_price = series.iloc[0]
    end_price = series.iloc[-1]
    value_factor = end_price / start_price
    year_past = get_years(series)
    return (value_factor ** (1 / year_past)) - 1

In [9]:
#9、构建夏普比率函数。
def sharpe_ratio(series:pd.Series,rf=0.015):
    return (calculate_cagr(series)-rf)/volatility(series)

In [10]:
#10、构建最大回撤函数。
def max_drawdown(series:pd.Series):
    previous_peaks=series.cummax()
    drawdown=-(series-previous_peaks)/previous_peaks
    return drawdown.max()

In [11]:
#11、构建Calmar比率函数。
def calmar_ratio(series:pd.Series):
    return cagr(series)/max_drawdown(series)

In [12]:
#12、构造再平衡函数。
#计算每12个月的累计收益率（每年第一个月重新计算，即cumprod的对象为每年的12个月）。
dataset_return=dataset_monthly.pct_change().dropna()
return_12m=(dataset_return+1).groupby(dataset_return.index.year).cumprod()
#将累计收益率矩阵第一行设定为1，方便矩阵相乘。
return_12m.loc[dataset_monthly.index[0]]=1
return_12m=return_12m.sort_index()
#制作一个矩阵，其每年各月投资额为上年年末投资用目标配比再平衡的结果，用其与上述每12个月累计收益率相乘，即为再平衡后的投资结果。
def rebalanced_invest_normed(dataset:pd.DataFrame,weight):
    #将资产价格标准化，方便与资产配比矩阵相乘，得出单位投资（1元）的回报值。
    dataset_norm=dataset/np.array(dataset.iloc[0,:])*weight
    dataset_copy=dataset_norm.copy()
    for year in range(round(get_years(dataset_norm))+1):
        dataset_copy.loc[f"{2005+year}",:]=float(dataset_norm.loc[f"{2004+year}-12"].sum(axis=1))*weight
    #计算每年再平衡后的投资结果
    return (dataset_copy.shift()*return_12m).sum(axis=1).dropna()

In [13]:
#13、构建投资组合并计算其再平衡后收益。
weight_1=np.array([0.4,0.6])
weight_2=np.array([0.3,0.7])
weight_3=np.array([0.2,0.8])
portfolio_1=rebalanced_invest_normed(dataset_monthly,weight_1)
portfolio_2=rebalanced_invest_normed(dataset_monthly,weight_2)
portfolio_3=rebalanced_invest_normed(dataset_monthly,weight_3)

In [14]:
#14、计算不同投资组合的波动率大小。
#设定计算的时间区间
day_1="2005-01-01"
day_2="2021-12-31"
vol_1=dataset_monthly[day_1:day_2].iloc[:,0].pct_change().std()
vol_2=dataset_monthly[day_1:day_2].iloc[:,1].pct_change().std()
vol_3=np.sqrt(np.dot(weight_1.T,np.dot(dataset_monthly[day_1:day_2].pct_change().cov(),weight_1)))
vol_4=np.sqrt(np.dot(weight_2.T,np.dot(dataset_monthly[day_1:day_2].pct_change().cov(),weight_2)))
vol_5=np.sqrt(np.dot(weight_3.T,np.dot(dataset_monthly[day_1:day_2].pct_change().cov(),weight_3)))
print("中证500在{}至{}间的波动率为".format(day_1,day_2),"%.2f"% (vol_1*100),"%")
print("上证国债在{}至{}间的波动率为".format(day_1,day_2),"%.2f"% (vol_2*100),"%")
print("4股6债在{}至{}间的波动率为".format(day_1,day_2),"%.2f"% (vol_3*100),"%")
print("3股7债在{}至{}间的波动率为".format(day_1,day_2),"%.2f"% (vol_4*100),"%")
print("2股8债在{}至{}间的波动率为".format(day_1,day_2),"%.2f"% (vol_5*100),"%")

中证500在2005-01-01至2021-12-31间的波动率为 8.31 %
上证国债在2005-01-01至2021-12-31间的波动率为 0.45 %
4股6债在2005-01-01至2021-12-31间的波动率为 3.27 %
3股7债在2005-01-01至2021-12-31间的波动率为 2.44 %
2股8债在2005-01-01至2021-12-31间的波动率为 1.62 %


In [15]:
#15、计算不同投资组合的累计收益率大小。
cum_1=cumulative_return(dataset_monthly[day_1:day_2].iloc[:,0])
cum_2=cumulative_return(dataset_monthly[day_1:day_2].iloc[:,1])
cum_3=cumulative_return(portfolio_1[day_1:day_2])
cum_4=cumulative_return(portfolio_2[day_1:day_2])
cum_5=cumulative_return(portfolio_3[day_1:day_2])
print("中证500在{}至{}间的累计收益率为".format(day_1,day_2),"%.2f"% (cum_1*100),"%")
print("上证国债在{}至{}间的累计收益率为".format(day_1,day_2),"%.2f"% (cum_2*100),"%")
print("4股6债在{}至{}间的累计收益率为".format(day_1,day_2),"%.2f"% (cum_3*100),"%")
print("3股7债在{}至{}间的累计收益率为".format(day_1,day_2),"%.2f"% (cum_4*100),"%")
print("2股8债在{}至{}间的累计收益率为".format(day_1,day_2),"%.2f"% (cum_5*100),"%")

中证500在2005-01-01至2021-12-31间的累计收益率为 465.07 %
上证国债在2005-01-01至2021-12-31间的累计收益率为 96.92 %
4股6债在2005-01-01至2021-12-31间的累计收益率为 241.92 %
3股7债在2005-01-01至2021-12-31间的累计收益率为 205.40 %
2股8债在2005-01-01至2021-12-31间的累计收益率为 169.07 %


In [16]:
#16、计算不同投资组合的复合年化收益率大小。
cagr_1=cagr(dataset_monthly[day_1:day_2].iloc[:,0])
cagr_2=cagr(dataset_monthly[day_1:day_2].iloc[:,1])
cagr_3=cagr(portfolio_1[day_1:day_2])
cagr_4=cagr(portfolio_2[day_1:day_2])
cagr_5=cagr(portfolio_3[day_1:day_2])
print("中证500在{}至{}间的复合年化收益率为".format(day_1,day_2),"%.2f"% (cagr_1*100),"%")
print("上证国债在{}至{}间的复合年化收益率为".format(day_1,day_2),"%.2f"% (cagr_2*100),"%")
print("4股6债在{}至{}间的复合年化收益率为".format(day_1,day_2),"%.2f"% (cagr_3*100),"%")
print("3股7债在{}至{}间的复合年化收益率为".format(day_1,day_2),"%.2f"% (cagr_4*100),"%")
print("2股8债在{}至{}间的复合年化收益率为".format(day_1,day_2),"%.2f"% (cagr_5*100),"%")

中证500在2005-01-01至2021-12-31间的复合年化收益率为 10.78 %
上证国债在2005-01-01至2021-12-31间的复合年化收益率为 4.09 %
4股6债在2005-01-01至2021-12-31间的复合年化收益率为 7.54 %
3股7债在2005-01-01至2021-12-31间的复合年化收益率为 6.82 %
2股8债在2005-01-01至2021-12-31间的复合年化收益率为 6.03 %


In [17]:
#17、计算不同投资组合的夏普比率大小。
print("中证500在{}至{}间的夏普比率为".format(day_1,day_2),"%.2f"% ((cagr_1-rf)/vol_1))
print("上证国债在{}至{}间的夏普比率为".format(day_1,day_2),"%.2f"% ((cagr_2-rf)/vol_2))
print("4股6债在{}至{}间的夏普比率为".format(day_1,day_2),"%.2f"% ((cagr_3-rf)/vol_3))
print("3股7债在{}至{}间的夏普比率为".format(day_1,day_2),"%.2f"% ((cagr_4-rf)/vol_4))
print("2股8债在{}至{}间的夏普比率为".format(day_1,day_2),"%.2f"% ((cagr_5-rf)/vol_5))

中证500在2005-01-01至2021-12-31间的夏普比率为 1.12
上证国债在2005-01-01至2021-12-31间的夏普比率为 5.70
4股6债在2005-01-01至2021-12-31间的夏普比率为 1.85
3股7债在2005-01-01至2021-12-31间的夏普比率为 2.18
2股8债在2005-01-01至2021-12-31间的夏普比率为 2.79


In [18]:
#18、计算不同投资组合的Calmar比率大小。
calmar_1=calmar_ratio(dataset_monthly[day_1:day_2].iloc[:,0])
calmar_2=calmar_ratio(dataset_monthly[day_1:day_2].iloc[:,1])
calmar_3=calmar_ratio(portfolio_1[day_1:day_2])
calmar_4=calmar_ratio(portfolio_2[day_1:day_2])
calmar_5=calmar_ratio(portfolio_3[day_1:day_2])
print("中证500在{}至{}间的Calmar比率为".format(day_1,day_2),"%.2f"% calmar_1)
print("上证国债在{}至{}间的Calmar比率为".format(day_1,day_2),"%.2f"% calmar_2)
print("4股6债在{}至{}间的Calmar比率为".format(day_1,day_2),"%.2f"% calmar_3)
print("3股7债在{}至{}间的Calmar比率为".format(day_1,day_2),"%.2f"% calmar_4)
print("2股8债在{}至{}间的Calmar比率为".format(day_1,day_2),"%.2f"% calmar_5)

中证500在2005-01-01至2021-12-31间的Calmar比率为 0.15
上证国债在2005-01-01至2021-12-31间的Calmar比率为 2.09
4股6债在2005-01-01至2021-12-31间的Calmar比率为 0.15
3股7债在2005-01-01至2021-12-31间的Calmar比率为 0.16
2股8债在2005-01-01至2021-12-31间的Calmar比率为 0.19
